In [2]:
import xml.etree.ElementTree as ET
import os
import numpy as np

In [3]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

In [4]:
def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [5]:
def read_classnames(classname):
    return [line.split(' ')[0][:-1] for line in open(r'E:\dataset\Animal\imagenet\\' + classname + '_wnids.txt').readlines()]

In [6]:
def extract_wnid(imagename):
    return imagename[:imagename.rfind('_')]

In [7]:
def extract_image_id(full_id):
    ind = full_id.rindex('\\')
    return full_id[ind+1:-1]
    

In [8]:
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             ovthresh=0.5,
             use_07_metric=False):
    """rec, prec, ap = voc_eval(detpath,
                                annopath,
                                imagesetfile,
                                classname,
                                [ovthresh],
                                [use_07_metric])
    Top level function that does the PASCAL VOC evaluation.
    detpath: Path to detections
        detpath.format(classname) should produce the detection results file.
    annopath: Path to annotations
        annopath.format(wnid,imagename) should be the xml annotations file.
    imagesetfile: Text file containing the list of images, one image per line.
    classname: Category name (duh)
    cachedir: Directory for caching the annotations
    [ovthresh]: Overlap threshold (default = 0.5)
    [use_07_metric]: Whether to use VOC07's 11 point AP computation
        (default False)
    """
    # assumes detections are in detpath.format(classname)
    # assumes annotations are in annopath.format(imagename)
    # assumes imagesetfile is a text file with each line an image name
    # cachedir caches the annotations in a pickle file

    #read classnames in  ImagenetFormat
    imagenet_classnames = read_classnames(classname)
    
    # first load gt

    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]

    
    # load annots
    recs = {}
    for i, imagename in enumerate(imagenames):
        recs[imagename] = parse_rec(annopath.format(extract_wnid(imagename),imagename))
        #if i % 100 == 0:
            #print 'Reading annotation for {:d}/{:d}'.format(i + 1, len(imagenames))
    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] in imagenet_classnames]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()
    if len(lines)<=0:
        return None,None,None
    
    splitlines = [x.strip().split(' ') for x in lines]
    image_ids = [extract_image_id(repr(x[0])) for x in splitlines]
    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d, :].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
            # compute overlaps
            # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
            if not R['difficult'][jmax]:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
        else:
            fp[d] = 1.

    # compute precision recall
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)
    
    return rec, prec, ap

In [10]:
import matplotlib.pyplot  as plt
from pylab import *
import numpy as np
from collections import defaultdict  

classnames = ['person', 'animal']
results = {}
mAPs = defaultdict(list)
for iteration in range(1,51):
    print ('iter: {}'.format(iteration))
    for classname in classnames:
        results[classname] = voc_eval(r'C:\darknet\build\darknet\x64\results\comp4_det_test_{0}-animal17-' + str(iteration) + '000.txt',
                r'E:\dataset\Animal\imagenet\synsets\{0}\annotations\{1}.xml',
                r'E:\dataset\Animal\imagenet\imagesetfile_dev_val.txt',
                classname,use_07_metric=True)
    mAP = 0
    fig= plt.figure()
    count = 0
    
    for i,classname in enumerate(classnames):
        rec,prec,ap = results[classname]
        if rec is None:
            continue
        print (rec.shape,prec.shape)
        mAP+=ap
        ax = fig.add_subplot(221+i)  
        plt.title(classname)
        plt.plot(rec,label='recall')
        plt.plot(prec,label='precision')
        F1 = 2*(prec*rec)/(prec+rec)
        plt.plot(F1,label='F1 score')
        plt.legend()
        plt.grid()
        fig.subplots_adjust(hspace=.5)
        
        mAPs[classname].append(ap)
    mAP = mAP/len(classnames)
    ax = fig.add_subplot(224)
    plt.title('mAP')
    ax.bar(1,mAP,0.05)
    ax.text(1,0.6,'%.2f'%(mAP),color='red')
    ax.set_yticks(np.linspace(0,1,11))
    #ax.set_yticks(np.linspace(0,1,11))
    savefig(r'C:\darknet\build\darknet\x64\eval\animal17\animal17_evaluation_{}.jpg'.format(iteration),dpi=200)
    #plt.show()


iter: 1


C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


(5945,) (5945,)
(9849,) (9849,)
iter: 2
(63309,) (63309,)
(85913,) (85913,)
iter: 3
(47117,) (47117,)
(64590,) (64590,)
iter: 4
(37163,) (37163,)
(57407,) (57407,)
iter: 5
(36477,) (36477,)
(57631,) (57631,)
iter: 6
(28890,) (28890,)
(50130,) (50130,)
iter: 7
(28187,) (28187,)
(45073,) (45073,)
iter: 8
(26051,) (26051,)
(39960,) (39960,)
iter: 9
(23015,) (23015,)
(43610,) (43610,)
iter: 10
(22056,) (22056,)
(42932,) (42932,)
iter: 11
(22709,) (22709,)
(35765,) (35765,)
iter: 12
(21125,) (21125,)
(41539,) (41539,)
iter: 13
(21255,) (21255,)
(33782,) (33782,)
iter: 14
(19151,) (19151,)
(36691,) (36691,)
iter: 15
(19294,) (19294,)
(31537,) (31537,)
iter: 16
(18244,) (18244,)
(35442,) (35442,)
iter: 17
(17510,) (17510,)
(30508,) (30508,)
iter: 18
(15557,) (15557,)
(36129,) (36129,)
iter: 19
(17273,) (17273,)
(31554,) (31554,)
iter: 20
(17684,) (17684,)
(32428,) (32428,)
iter: 21
(19919,) (19919,)
(36240,) (36240,)
iter: 22
(16382,) (16382,)
(29416,) (29416,)
iter: 23
(14638,) (14638,)
(298

In [11]:
for cls in classnames: 
    fig= plt.figure()
    plt.plot(mAPs[cls])
    plt.ylabel('mean Average Precision')
    plt.xlabel('number of iterations in 1000s')
    plt.yticks(np.linspace(0,1,11))
    plt.grid()
    savefig(r'C:\darknet\build\darknet\x64\eval\animal17\evaluation_{}.png'.format(cls),dpi=200)
#plt.show()

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
print 